# Домашнее задание к лекции «Улучшение качества модели»


## Задание

#### Цель: Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения

#### Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов. Данные для обучения моделей необходимо загрузить с [сайта](https://www.kaggle.com/fedesoriano/heart-failure-prediction). Целевая переменная – наличие болезни сердца (HeartDisease), принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. (Подробнее о признаках можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.)

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [76]:
import warnings
warnings.filterwarnings('ignore')

## Этапы работы:

### 1. Получите данные и загрузите их в рабочую среду

In [77]:
my_path = r"C:\Users\sveta\Documents\Netology\ML\heart.csv"
data = pd.read_csv(my_path)
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


Attribute Information:
    
- Age: возраст пациента [лет]
- Sex: пол пациента [M: Male, F: Female]
- ChestPainType: тип боли в груди [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
- RestingBP: артериальное давление в состоянии покоя [mm Hg]
- Cholesterol: сывороточный холестерин [mm/dl]
- FastingBS: уровень сахара в крови натощак [1: if FastingBS > 120 mg/dl, 0: otherwise]
- RestingECG: результаты электрокардиограммы в покое [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
- MaxHR: максимальная достигнутая частота сердечных сокращений [Numeric value between 60 and 202]
- ExerciseAngina: стенокардия, вызванная физической нагрузкой [Y: Yes, N: No]
- Oldpeak: oldpeak = ST [Numeric value measured in depression]
- ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
- HeartDisease: output class [1: heart disease, 0: Normal]

### 2. Подготовьте датасет к обучению моделей.

#### a) Категориальные переменные переведите в цифровые значения

In [79]:
categorials = data.select_dtypes('object').columns
categorials

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')

In [80]:
dummy_data = pd.get_dummies(data, columns=categorials, drop_first= True)
dummy_data.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,1,0,0,0,1


### 3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [81]:
from sklearn.model_selection import train_test_split

In [82]:
X = dummy_data.drop(columns=['HeartDisease'])
y = dummy_data[['HeartDisease']]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

### 4. Обучите модель логистической регрессии с параметрами по умолчанию

In [84]:
from sklearn.linear_model import LogisticRegression

Parameters:

- penalty{'l1', 'l2', 'elasticnet', 'none'}, default='l2'
Используется для указания нормы, используемой в наказании. Решатели «newton-cg», «sag» и «lbfgs» поддерживают только штрафы l2. 
- dual bool, default=False
Двойная или первичная формулировка. Предпочитать dual = False, когда n_samples > n_features.

- tol float, default=1e-4
Толерантность к критериям остановки.

- **C** float, default=1.0
Обратная сила регуляризации; должен быть положительный. Как и в векторных машинах поддержки, меньшие значения указывают более сильную регуляризацию.

- fit_interceptbool, default=True
Указывает, следует ли добавлять к функции принятия решения константу (так же известную как смещение или перехват).

- intercept_scalingfloat, default=1
Полезно, только когда используется решатель liblinear и для self.fit_intercept установлено значение True. В этом случае x становится [x, self.intercept_scaling], т. е. к вектору экземпляра добавляется «синтетический» признак с постоянным значением, равным intercept_scaling. Перехват становится intercept_scaling * synthetic_feature_weight .

- class_weightdict or 'balanced', default=None
Веса, связанные с классами в форме {class_label: weight} . Если не указано иное, все классы должны иметь вес один.

«Сбалансированный» режим использует значения y для автоматической корректировки весов, обратно пропорциональных частотам классов во входных данных, как n_samples / (n_classes * np.bincount(y)) .

- **random_state** int, RandomState instance, default=None
Используется, когда solver == 'sag', 'saga' или 'liblinear' для перемешивания данных. 

- **solver**{'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}, default='lbfgs'

- **max_iterint**, default=100
Максимальное количество итераций,необходимое для сходимости решателей.

- multi_class{'auto', 'ovr', 'multinomial'}, default='auto'

- verboseint, default=0
Для губчатых и lbfgs solvers установите любое положительное число для глаголов.

- n_jobs int, default=None
Количество ядер ЦП, используемых при распараллеливании по классам, если multi_class='ovr'. 


In [85]:
logreg = LogisticRegression()

# Обучаем модель на тренировочных данных
logreg.fit(X_train, y_train)

# Предсказываем значения для тестовых данных
y_pred = logreg.predict(X_test)

### 5. Подсчитайте основные метрики модели
Используйте следующие метрики и `функцию:cross_validate(…, cv=10, scoring=['accuracy','recall','precision','f1'])`

In [86]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [87]:
scoring = ['accuracy', 'recall', 'precision', 'f1']

# Применение функции cross_validate для оценки модели
cv_results = cross_validate(logreg, X, y.values.ravel(), cv=10, scoring=scoring)

print("Средние значения метрик:")
print("Accuracy:", cv_results['test_accuracy'].mean())
print("Recall:", cv_results['test_recall'].mean())
print("Precision:", cv_results['test_precision'].mean())
print("F1-score:", cv_results['test_f1'].mean())


Средние значения метрик:
Accuracy: 0.8516483516483516
Recall: 0.8695294117647057
Precision: 0.8682864966913024
F1-score: 0.8644385680123691


### 6. Оптимизируйте 3-4 параметра модели:

##### Изменим обратную силу регуляризации

In [88]:
logreg1 = LogisticRegression(C=0.01)
logreg1.fit(X_train, y_train)
y_pred1 = logreg1.predict(X_test)

In [89]:
scoring = ['accuracy', 'recall', 'precision', 'f1']

# Применение функции cross_validate для оценки модели
cv_results = cross_validate(logreg1, X, y.values.ravel(), cv=10, scoring=scoring)

print("Средние значения метрик:")
print("Accuracy:", cv_results['test_accuracy'].mean())
print("Recall:", cv_results['test_recall'].mean())
print("Precision:", cv_results['test_precision'].mean())
print("F1-score:", cv_results['test_f1'].mean())


Средние значения метрик:
Accuracy: 0.8147037744863832
Recall: 0.836156862745098
Precision: 0.8382347442704144
F1-score: 0.8300343479794062


In [90]:
logreg1 = LogisticRegression(C=100)
logreg1.fit(X_train, y_train)
y_pred1 = logreg1.predict(X_test)

In [91]:
scoring = ['accuracy', 'recall', 'precision', 'f1']

# Применение функции cross_validate для оценки модели
cv_results = cross_validate(logreg1, X, y.values.ravel(), cv=10, scoring=scoring)

print("Средние значения метрик:")
print("Accuracy:", cv_results['test_accuracy'].mean())
print("Recall:", cv_results['test_recall'].mean())
print("Precision:", cv_results['test_precision'].mean())
print("F1-score:", cv_results['test_f1'].mean())


Средние значения метрик:
Accuracy: 0.8494744386048735
Recall: 0.8636078431372548
Precision: 0.8680930758348879
F1-score: 0.8617829832743682


Любые значительные отклонения от С = 1 ухудшают метрики модели

##### Изменим random_state

In [92]:
from sklearn.model_selection import GridSearchCV

In [93]:
param_grid = {
    'random_state': [11, 42, 83, 1004]
}

In [94]:
logreg2 = LogisticRegression()

In [95]:
grid_search = GridSearchCV(logreg, param_grid, scoring=['accuracy', 'recall', 'precision', 'f1'], refit=False, cv=10)
grid_search.fit(X_train, y_train.values.ravel())

# Получение результатов 
cv_results = grid_search.cv_results_

In [96]:
for i, random_state in enumerate(param_grid['random_state']):
    print("Значение random_state:", random_state)
    print("Средние значения метрик:")
    print("Accuracy:", cv_results['mean_test_accuracy'][i])
    print("Recall:", cv_results['mean_test_recall'][i])
    print("Precision:", cv_results['mean_test_precision'][i])
    print("F1-score:", cv_results['mean_test_f1'][i])
    print()

Значение random_state: 11
Средние значения метрик:
Accuracy: 0.8623102554609403
Recall: 0.8899390243902439
Precision: 0.8675546584663948
F1-score: 0.8779082610960799

Значение random_state: 42
Средние значения метрик:
Accuracy: 0.8623102554609403
Recall: 0.8899390243902439
Precision: 0.8675546584663948
F1-score: 0.8779082610960799

Значение random_state: 83
Средние значения метрик:
Accuracy: 0.8623102554609403
Recall: 0.8899390243902439
Precision: 0.8675546584663948
F1-score: 0.8779082610960799

Значение random_state: 1004
Средние значения метрик:
Accuracy: 0.8623102554609403
Recall: 0.8899390243902439
Precision: 0.8675546584663948
F1-score: 0.8779082610960799



Очевидно, random_state не влияет на результат

##### Изменим алгоритм оптимизации функции потерь

In [97]:
param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

In [98]:
logreg3 = LogisticRegression()
grid_search = GridSearchCV(logreg, param_grid, scoring=['accuracy', 'recall', 'precision', 'f1'], refit=False, cv=10)
grid_search.fit(X_train, y_train.values.ravel())

cv_results = grid_search.cv_results_

In [99]:
for i, solver in enumerate(param_grid['solver']):
    print("Значение solver:", solver)
    print("Средние значения метрик:")
    print("Accuracy:", cv_results['mean_test_accuracy'][i])
    print("Recall:", cv_results['mean_test_recall'][i])
    print("Precision:", cv_results['mean_test_precision'][i])
    print("F1-score:", cv_results['mean_test_f1'][i])
    print()

Значение solver: newton-cg
Средние значения метрик:
Accuracy: 0.8595520177711959
Recall: 0.884939024390244
Precision: 0.8667826636963426
F1-score: 0.8750776231471618

Значение solver: lbfgs
Средние значения метрик:
Accuracy: 0.8623102554609403
Recall: 0.8899390243902439
Precision: 0.8675546584663948
F1-score: 0.8779082610960799

Значение solver: liblinear
Средние значения метрик:
Accuracy: 0.8582006664198445
Recall: 0.884939024390244
Precision: 0.8649469149040719
F1-score: 0.8740618519629862

Значение solver: sag
Средние значения метрик:
Accuracy: 0.7411514253980007
Recall: 0.7922560975609756
Precision: 0.7559020583686313
F1-score: 0.7726275744922412

Значение solver: saga
Средние значения метрик:
Accuracy: 0.7180118474639021
Recall: 0.7654268292682926
Precision: 0.7389036124573558
F1-score: 0.7504011404347353



Лучше всего себя показал L-BFGS (Limited-memory Broyden-Fletcher-Goldfarb-Shanno), который стоит по умолчанию. 
Хуже всех справился saga (стохастический средний градиент, который также поддерживает L1-регуляризацию). Скорее всего, для него слишком мало данных.

##### Изменим количество итераций

In [100]:
param_grid = {
    'max_iter': [10, 200, 1000]
}

In [101]:
logreg4 = LogisticRegression()
grid_search = GridSearchCV(logreg, param_grid, scoring=['accuracy', 'recall', 'precision', 'f1'], refit=False, cv=10)
grid_search.fit(X_train, y_train.values.ravel())

cv_results = grid_search.cv_results_

In [102]:
for i, max_iter in enumerate(param_grid['max_iter']):
    print("Значение max_iter:", max_iter)
    print("Средние значения метрик:")
    print("Accuracy:", cv_results['mean_test_accuracy'][i])
    print("Recall:", cv_results['mean_test_recall'][i])
    print("Precision:", cv_results['mean_test_precision'][i])
    print("F1-score:", cv_results['mean_test_f1'][i])
    print()

Значение max_iter: 10
Средние значения метрик:
Accuracy: 0.6907441688263607
Recall: 0.7482317073170732
Precision: 0.7130423155710462
F1-score: 0.7293189532962455

Значение max_iter: 200
Средние значения метрик:
Accuracy: 0.8622917437985931
Recall: 0.8873780487804878
Precision: 0.8694956012809104
F1-score: 0.8775917606206487

Значение max_iter: 1000
Средние значения метрик:
Accuracy: 0.8582006664198445
Recall: 0.884939024390244
Precision: 0.8649469149040719
F1-score: 0.8740618519629862



Для данной модели число операций ~100 является оптимальным, любые изменения ухудшают метрики

### 7. Сформулируйте выводы по проделанной работе:

#### a) Сравните метрики построенных моделей

Для данной небольшой выборки данных оказалось достаточно стандартных параметров логистической регрессии. Серьезные изменения приводят к снижению качества модели. 

| Метрика | Среднее значение |
|:------------------------|---------|
|Accuracy| 0.85|
|Recall| 0.87|
|Precision| 0.87|
|F1-score| 0.86|

#### b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

Выводы работы по ансамблированию:

#### a) Сравните метрики построенных моделей.

| Модель                         | Precision | Recall  | F1-score | Support |
|:--------------------------------|-----------|---------|----------|---------|
| Decision Tree Classifier      | 0.78      | 0.74    | 0.76     | 99      |
| Random Forest Classifier      | 0.87     | 0.91    | 0.89     | 99      |
| Bagging Classifier (Decision Tree) | 0.88      | 0.85    | 0.86     | 99     |
| Stacking Classifier | 0.84      | 0.91    | 0.87     | 99     |

#### b) Напишите свое мнение, какая модель наилучшая и почему.

Лучше всех моделей наличие болезни определяет модель Random Forest Classifier. Она позволяет выявить важность признаков и использовать наиболее значимые. Однако деревья склонны к переобучению. Поэтому наилучшей моделью является Stacking Classifier. Она использует достоинства случайного леса, но более гибкая 

По сравнению с деревьями, модель логистической регрессии менее склонна к переобучение. Ее метрики не так разнятся, хотя специфика задачи предполагает классификацию, регрессия справляется неплохо. Модель можно улучшить, используя комбинации различных параметров.